In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

In [2]:
path='/mlProjects/imageCol/'
images = []
Names = []
imagesList = os.listdir('/mlProjects/imageCol/')

for img in imagesList:
    curImg = cv2.imread(f'{path}/{img}')
    images.append(curImg)
    Names.append(os.path.splitext(img)[0])
print(Names)
 

['Bill Gates', 'Elon Musk', 'Lionel Messi', 'Ratan Tata']


In [3]:
def findEncodings(images):
    encodedFaces = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodedFaces.append(encode)
    return encodedFaces
encodeList = findEncodings(images)

In [4]:
def markAttendance(name):
    with open('/mlProjects/attendancefile.csv','r+') as f:
        dataList = f.readlines()
        nameList = []
        for line in dataList:
            atten = line.split(',')
            nameList.append(atten[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')

In [ ]:
cap = cv2.VideoCapture(0)
 
while True:
    success, img = cap.read()
    #img = captureScreen()
    imgSmall = cv2.resize(img,(0,0),None,0.25,0.25)
    imgSmall = cv2.cvtColor(imgSmall, cv2.COLOR_BGR2RGB)

    facesinFrame = face_recognition.face_locations(imgSmall)
    encodesinFrame = face_recognition.face_encodings(imgSmall,facesinFrame)

    for encodeFace,faceLoc in zip(encodesinFrame,facesinFrame):
            matches = face_recognition.compare_faces(encodeList,encodeFace)
            faceDis = face_recognition.face_distance(encodeList,encodeFace)
           
            matchIndex = np.argmin(faceDis)

            if matches[matchIndex]:
                name = Names[matchIndex].upper()
                #print(name)
                y1,x2,y2,x1 = faceLoc
                y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
                cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
                cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
                cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                markAttendance(name)

    cv2.imshow('Webcam',img)
    cv2.waitKey(0)